# How to solve systems of linear equations on a near-term quantum computer

In this notebook, you will be using [PennyLane](https://pennylane.ai/) to implement and run the Variational Quantum Linear Solver (VQLS) algorithm with the goal to solve an $8\times8$ system of linear equations. Later, you will run your PennyLane code on [Amazon Braket](https://aws.amazon.com/braket/) in the cloud to solve a larger system of size $1024\times1024$.

## References

* A detailed description of the algorithm can be found in the [original publication of the Variational Quantum Linear Solver](https://arxiv.org/abs/1909.05820)
* This notebook closely follows the reference implementation in PennyLane published in [this tutorial of the Variational Quantum Linear Solver](https://pennylane.ai/qml/demos/tutorial_vqls/)
* Check out the Amazon Braket developer guide to learn how you can [use PennyLane with Amazon Braket](https://docs.aws.amazon.com/braket/latest/developerguide/hybrid.html)


## Background

In many applications in science and technology, we are interested in finding a solution for a [system of $N$ linear equations](https://en.wikipedia.org/wiki/System_of_linear_equations):

$$
\sum\limits_{j=1}^{N} a_{1j} x_j = b_1 \\
\vdots \\
\sum\limits_{j=1}^{N} a_{Nj} x_j = b_N
$$

where $x_1,\cdots,x_N$ are the unknown variables to be solved for and $a_{ij}$ and $b_1,\cdots,b_N$ are known coefficients. If we arrange $a_{ij}$ in an $N\times N$ matrix $A$, $x_i$ and $b_i$ into column vectors ${\bf x}$ and ${\bf b}$ then we can rewrite the linear system above as $$A{\bf x}={\bf b}.$$

Systems of this type are frequently encountered when solving differential equations, performing linear regression and curve-fitting, in various combinatorial applications, and in machine learning.

## Quantum vs classical

Since systems of linear equations are encountered in physics, chemistry, differential equations, and other areas many classical techniques have been developed to solve them. One of the oldest is [Cramer's rule](https://en.wikipedia.org/wiki/Cramer%27s_rule), which is inefficient because it requires computing many [matrix determinants](https://en.wikipedia.org/wiki/Determinant). Another is [Gaussian elimination](https://en.wikipedia.org/wiki/Gaussian_elimination), which only needs one determinant to be computed, and has a complexity of $O(N^3)$ where $N$ is the number of equations. Beyond these techniques there are many "numerical" (e.g. computational) methods for these problems. There is a whole field, [linear programming](https://en.wikipedia.org/wiki/Linear_programming) devoted to solving them because of their many applications in transportation planning, logistics, and other optimization problems. The classical complexity of solving a system of linear equations can depend on precise details of $A$, the matrix encoding the problem, but in the worst case techniques like Gaussian elimination with pivoting are used, which are $O(N^3)$. Alternatively, an initial solution is proposed and then iteratively improved upon using the diagonal and off-diagonal components -- this is the [Jacobi method](https://en.wikipedia.org/wiki/Jacobi_method) but cannot be used for arbitrary $A$. For large $N$ this is unfavorable and so a quantum algorithm with better scaling would be attractive for such tasks.

## Example problem

Let us start with a concrete example and look at the following system of $8$ linear equations:

$$
\begin{pmatrix}
1   & 0   & 0 & 0 & 0.4 & 0   & 0 & 0 \\
0   & 1   & 0 & 0 & 0   & 0.4 & 0 & 0 \\
0   & 0   & 1 & 0 & 0   & 0   & 0 & 0 \\
0   & 0   & 0 & 1 & 0   & 0   & 0 & 0 \\
0.4 & 0   & 0 & 0 & 1   & 0   & 0 & 0 \\ 
0   & 0.4 & 0 & 0 & 0   & 1   & 0 & 0 \\
0   & 0   & 0 & 0 & 0   & 0   & 1 & 0 \\
0   & 0   & 0 & 0 & 0   & 0   & 0 & 1
\end{pmatrix}
\begin{pmatrix}
x_1 \\
x_2 \\
x_3 \\
x_4 \\
x_5 \\ 
x_6 \\
x_7 \\
x_8   
\end{pmatrix} = \frac{1}{2\sqrt{2}}
\begin{pmatrix}
1 \\
1 \\
1 \\
1 \\
1 \\ 
1 \\
1 \\
1   
\end{pmatrix}
$$

## Classical solutions

To solve the problem classically, we can use the explicit matrix representation in terms of numerical [NumPy](https://numpy.org/devdocs/index.html) arrays:

In [ ]:
from pennylane import numpy as np

A = np.matrix([
    [1, 0, 0, 0, 0.4, 0, 0, 0],
    [0, 1, 0, 0, 0, 0.4, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0.4, 0, 0, 0, 1, 0, 0, 0],
    [0, 0.4, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 1],
])
print(f"A = {A}\n")

b = (0.5/np.sqrt(2))*np.matrix(np.ones([8, 1]))
print(f"b = {b}")

A convenient approach if the matrix $A$ is [invertible](https://en.wikipedia.org/wiki/Invertible_matrix) is to obtain an exact solution from its [inverse](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html) $A^{-1}$ with ${\bf x} = A^{-1}{\bf b}$:

In [ ]:
A_inv = np.matrix(np.linalg.inv(A))
x_inv = np.dot(A_inv, b)
print(x_inv)

Alternatively, we can find approximate solutions, for example using the approach of [least squares](https://en.wikipedia.org/wiki/Linear_least_squares) to find ${\bf x}$ by minimizing the norm $|\!|A{\bf x}-{\bf b}{|\!|}^{2} = {\bf x}^{T}A^{T}A{\bf x} - {\bf x}^{T}A^{T}{\bf b} - {\bf b}^{T}A{\bf x} + {\bf b}^{T}{\bf b} $

In [ ]:
x_ls = np.linalg.lstsq(A, b, rcond=None)[0]
print(x_ls)

## Brief (re-)introduction to quantum computing fundamentals

Expand the cell below if you want a refresher of what qubits and gates are and how they can be represented.

### Qubits and qubit states

A **qubit** (or quantum bit) is a basic unit of information used in quantum computers, equivalently to the classical bit in classical computers. It is a two-level quantum-mechanical system which can be represented as a 2-dimensional complex vector space. With the orthonormal basis states (or basis vectors) 

$$|0\rangle = \begin{pmatrix} 1 \\ 0 \end{pmatrix}, \; |1\rangle = \begin{pmatrix} 0 \\ 1 \end{pmatrix},$$ 

we can write the state of a qubit at a given time as a 2-dimensional complex vector, the so-called **state vector** 

$$|\psi\rangle = \alpha |0\rangle + \beta |1\rangle,$$ 

representing a superposition or linear combination of the basis states with amplitudes $\alpha$ and $\beta$ being complex numbers.

<div class="alert alert-block alert-success"> 
    <b>Note:</b> Throughout this notebook we use the <a href="https://en.wikipedia.org/wiki/Bra%E2%80%93ket_notation">Dirac notation</a> (or Bra-ket notation) which has been introduced to write the linear algebra expressions and operations of quantum mechanics in a compact way.
</div>

When we perform a **measurement** of the qubit in the standard or computational basis $\{|0\rangle,|1\rangle\}$ we retrieve a bit representation of the basis states according to the [Born rule](https://en.wikipedia.org/wiki/Born_rule): the probability of measuring the system in state $|0\rangle$ (corresponding to the bit "0") is $|\alpha|^2$ and the probability of $|1\rangle$ (corresponding to "1") is $|\beta|^2$ with $$|\alpha|^2 + |\beta|^2 = 1.$$

The possible states of a single qubit can be visualized using the [Bloch sphere](https://en.wikipedia.org/wiki/Bloch_sphere), a unit 2-sphere with the "north pole" and "south pole" corresponding to $|0\rangle$ and $|1\rangle$, respectively, and the amplitudes $\alpha$ and $\beta$ transformed to spherical coordinates with 

$$\alpha = \cos\left(\frac{\theta}{2}\right), \; \beta = e^{i\phi}\sin\left(\frac{\theta}{2}\right).$$ 

Any qubit state $|\psi\rangle$ can be represented as a point on the surface of the Bloch sphere.

<div align="center">
    <figure>
        <img src="images/Bloch_sphere.svg" width="350"/>
        <figcaption>Wikimedia Commons, <a href="https://commons.wikimedia.org/wiki/File:Bloch_sphere.svg">File:Bloch_sphere.svg</a></figcaption>
    </figure>
</div>

In quantum computing, we typically look at states of **multiple qubits** (also called a [quantum register](https://en.wikipedia.org/wiki/Quantum_register)). For example, the state vector for a two-qubit system is represented by the linear combination 

$$|\psi\rangle=\alpha|00\rangle+\beta|01\rangle+\gamma|10\rangle+\delta|11\rangle$$ 

with the basis states 

$$|00\rangle=|0\rangle_{1}\otimes|0\rangle_{2}=\begin{pmatrix}1\\0\\0\\0\end{pmatrix}, \; |01\rangle=|0\rangle_{1}\otimes|1\rangle_{2}=\begin{pmatrix}0\\1\\0\\0\end{pmatrix}, \; |10\rangle=|1\rangle_{1}\otimes|0\rangle_{2}=\begin{pmatrix}0\\0\\1\\0\end{pmatrix}, \; |11\rangle=|1\rangle_{1}\otimes|1\rangle_{2}=\begin{pmatrix}0\\0\\0\\1\end{pmatrix},$$ 

where $\otimes$ denotes the [Kronecker product](https://en.wikipedia.org/wiki/Kronecker_product) (also can be viewed as a vectorization of the [outer product](https://en.wikipedia.org/wiki/Outer_product#Connection_with_the_Kronecker_product)) of the single-qubit basis states.

In general, $n$ qubits are represented by a $2^n$-dimensional state vector 

$$|\psi\rangle =  x_0 |0\rangle_{1} \otimes \cdots \otimes |0\rangle_{n} + x_1 |0\rangle_{1} \otimes \cdots |0\rangle_{n-1} \otimes |1\rangle_{n} + \cdots + x_{2^{n}-1} |1\rangle_{1} \otimes \cdots \otimes |1\rangle_{n}.$$   

### Quantum operations
A **quantum gate** is a basic logic operation which evolves a qubit state. Quantum gates can be represented as unitary matrices. 

<div class="alert alert-block alert-success"> 
    <b>Reminder:</b> A complex square matrix $U$ is <b>unitary</b> if its inverse $U^{-1}$ is equal to its conjugate transpose $U^\dagger$, that is, if $U^\dagger U = U U^\dagger = \mathbb{I}$.
</div>

Let's look at some examples which may be helpful for the remainder of this workshop:

#### Single-qubit gates

The **identity gate** $\mathbb{I}=\begin{pmatrix}1 & 0 \\ 0 & 1\end{pmatrix}$ does not modify the state of a qubit $$I|\psi\rangle=|\psi\rangle$$

where $I|\psi\rangle$ denotes the the [inner product](https://en.wikipedia.org/wiki/Bra%E2%80%93ket_notation#Inner_product_and_bra%E2%80%93ket_identification_on_Hilbert_space) of $I$ and $|\psi\rangle$.

The **[Pauli](https://en.wikipedia.org/wiki/Pauli_matrices) gates** 

$$X = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}, \; Y = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}, \;  Z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix},$$ 

rotate the qubit state by $\pi$ radians around the $x$, $y$, and $z$ axes of the Bloch sphere. Together with the identity matrix $\mathbb{I}$, the Pauli matrices form a complete orthonormal basis for the space of single-qubit unitary matrices. The Pauli-$X$ gate is the quantum equivalent to the classical **NOT** gate as it flips the basis states 

$$X|0\rangle=|1\rangle , \; X|1\rangle=|0\rangle.$$

<div align="center">
    <figure>
        <img style="vertical-align:middle" src="images/zero.png" width="350"/>
        <img style="vertical-align:middle" src="images/arrow.png" width="100"/>
        <img style="vertical-align:middle" src="images/one.png" width="350"/>
    </figure>
</div>

<div align="center">
    <figure>
        <img style="vertical-align:middle" src="images/one.png" width="350"/>
        <img style="vertical-align:middle" src="images/arrow.png" width="100"/>
        <img style="vertical-align:middle" src="images/zero.png" width="350"/>
    </figure>
</div>

The **Hadamard gate** $H=\frac{1}{\sqrt{2}}\begin{pmatrix}1 & 1 \\ 1 & -1\end{pmatrix}$ evolves the computational basis states to an equal superposition:

$$H|0\rangle = \frac{|0\rangle + |1\rangle}{\sqrt{2}} = |+x\rangle, \; H|1\rangle = \frac{|0\rangle - |1\rangle}{\sqrt{2}} = |-x\rangle.$$ 

In other words, it rotates the $|0\rangle$ state to the $|+x\rangle$ state and the $|1\rangle$ state to the $|-x\rangle$ state, as shown on the two Bloch spheres below:

<div align="center">
    <figure>
        <img style="vertical-align:middle" src="images/zero.png" width="350"/>
        <img style="vertical-align:middle" src="images/arrow.png" width="100"/>
        <img style="vertical-align:middle" src="images/plus_x.png" width="350"/>
    </figure>
</div>

<div align="center">
    <figure>
        <img style="vertical-align:middle" src="images/one.png" width="350"/>
        <img style="vertical-align:middle" src="images/arrow.png" width="100"/>
        <img style="vertical-align:middle" src="images/neg_x.png" width="350"/>
    </figure>
</div>


Instead of rotating single qubits by fixed angles, we often deal with **parametrized rotation gates**, for example the gates

$$
R_x(\theta) = e^{-i\frac{\theta}{2}X} = \begin{pmatrix} \cos\left(\frac{\theta}{2}\right) & -i \sin\left(\frac{\theta}{2}\right) \\ -i \sin\left(\frac{\theta}{2}\right) & \cos\left(\frac{\theta}{2}\right) \end{pmatrix}, \;
R_y(\theta) = e^{-i\frac{\theta}{2}Y} = \begin{pmatrix} \cos\left(\frac{\theta}{2}\right) & -\sin\left(\frac{\theta}{2}\right) \\ \sin\left(\frac{\theta}{2}\right) & \cos\left(\frac{\theta}{2}\right) \end{pmatrix}, \;
R_z(\theta) = e^{-i\frac{\theta}{2}Z} = \begin{pmatrix} e^{-i\frac{\theta}{2}} & 0 \\ 0 & e^{i\frac{\theta}{2}}  \end{pmatrix},
$$ 

which rotate the qubit around the $X$, $Y$, and $Z$ axis of the Bloch sphere by an angle passed as a free parameter. For example, below we see an $R_x$ rotation by the angle $\frac{\pi}{4}$ of the two computational basis states visualized on the Bloch spheres below:

<div align="center">
    <figure>
        <img style="vertical-align:middle" src="images/zero.png" width="350"/>
        <img style="vertical-align:middle" src="images/arrow.png" width="100"/>
        <img style="vertical-align:middle" src="images/rx.png" width="350"/>
    </figure>
</div>

<div align="center">
    <figure>
        <img style="vertical-align:middle" src="images/one.png" width="350"/>
        <img style="vertical-align:middle" src="images/arrow.png" width="100"/>
        <img style="vertical-align:middle" src="images/neg_rx.png" width="350"/>
    </figure>
</div>

We can also generalize the $Z$ operating performing a phase flip by $\pi$ with the general **phase shift gate**

$$
R_\phi(\phi) = e^{i\frac{\phi}{2}}R_z(\phi) = \begin{pmatrix} 1 & 0 \\ 0 & e^{i\phi} \end{pmatrix}.
$$

#### Controlled gates

Controlled gates act on two or more qubits, where one more qubits act as a control for some operation on a single target qubit. For example, the **controlled NOT gate** (or CNOT or CX) acts on two qubits, and performs the NOT operation on the target qubit only when the control qubit is $|1\rangle$, and otherwise leaves it unchanged. With 

$$\text{CNOT}=\begin{pmatrix}1&0&0&0\\ 0&1&0&0\\ 0&0&0&1\\ 0&0&1&0\end{pmatrix}$$ we get $$\text{CNOT}|00\rangle=|00\rangle, \; \text{CNOT}|01\rangle=|01\rangle, \; \text{CNOT}|10\rangle=|11\rangle, \; \text{CNOT}|11\rangle=|10\rangle.$$

Generally, if $U$ is an arbitary single-qubit gate, the two-qubit controlled-$U$ gate operating such that the first qubit serves as the control and the second as the target qubit can be written as 

$$\text{C}U = |0\rangle \langle0| \otimes \mathbb{I} + |1\rangle \langle1| \otimes U =\begin{pmatrix}1 & 0 & \begin{matrix} 0 & 0 \end{matrix}\\ 0 & 1 & \begin{matrix} 0 & 0 \end{matrix} \\ \begin{matrix}0\\0\end{matrix} & \begin{matrix}0\\0\end{matrix} & U \end{pmatrix},$$

with the outer products $$|0\rangle \langle0| = \begin{pmatrix}1\\0\end{pmatrix}\begin{pmatrix}1&0\end{pmatrix} = \begin{pmatrix}1&0\\0&0\end{pmatrix}, \; |1\rangle \langle1| = \begin{pmatrix}0\\1\end{pmatrix}\begin{pmatrix}0&1\end{pmatrix} = \begin{pmatrix}0&0\\0&1\end{pmatrix}$$

and the Kronecker product $\otimes$ you have met before.

## Quantum solution

### Mapping the problem of systems of linear systems onto quantum computers

Quantum computers operate on quantum states of a qubit register. A quantum state of $n$ qubits can be mapped on a normalized $2^n$-dimensional vector of complex numbers and vice versa. For example, we can map the (normalized) $8$-dimensional vector of variables $[x_0,x_1, x_2, x_3,x_4,x_5,x_6,x_7]$ onto the quantum state of three qubits

$$|x\rangle = x_0 |000\rangle + x_1 |001\rangle + x_2 |010\rangle + x_3 |011\rangle + x_4 |100\rangle + x_5 |101\rangle + x_6 |110\rangle + x_7 |111\rangle$$

<div class="alert alert-block alert-success"> 
    Remember that you can review some fundamentals about qubits and gates in the <a href="#Brief-(re-)introduction-to-quantum-computing-fundamentals">previous section</a> of this notebook, including their standard vector and matrix representations, as well as information about the $|x\rangle$ notation we use here.
</div>

For the matrix $A$, we know that operations on quantum states of qubits are represented by unitary operators (see above). If the matrix $A$ is unitary, then it can be directly implemented as a quantum circuit - a series of quantum gates applied to the qubits. We can relax this constraint and require that $A$ is a $2^n \times 2^n$ matrix, which can be expressed as a linear combination of $L$ unitary matrices $A_l$ of the form $$A = \sum\limits_{l = 1}^{L}c_l A_l,$$ where $c_l$ are complex numbers.

Taking our example problem defined above, 
$$
\begin{pmatrix}
1   & 0   & 0 & 0 & 0.4 & 0   & 0 & 0 \\
0   & 1   & 0 & 0 & 0   & 0.4 & 0 & 0 \\
0   & 0   & 1 & 0 & 0   & 0   & 0 & 0 \\
0   & 0   & 0 & 1 & 0   & 0   & 0 & 0 \\
0.4 & 0   & 0 & 0 & 1   & 0   & 0 & 0 \\ 
0   & 0.4 & 0 & 0 & 0   & 1   & 0 & 0 \\
0   & 0   & 0 & 0 & 0   & 0   & 1 & 0 \\
0   & 0   & 0 & 0 & 0   & 0   & 0 & 1
\end{pmatrix}
\begin{pmatrix}
x_1 \\
x_2 \\
x_3 \\
x_4 \\
x_5 \\ 
x_6 \\
x_7 \\
x_8   
\end{pmatrix} = \frac{1}{2\sqrt{2}}
\begin{pmatrix}
1 \\
1 \\
1 \\
1 \\
1 \\ 
1 \\
1 \\
1   
\end{pmatrix}
$$ 
we can write the $8 \times 8$ matrix as a linear combination $$A = \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} + 0.2 X\otimes Z \otimes \mathbb{I} + 0.2 X\otimes \mathbb{I} \otimes \mathbb{I}$$ of Pauli and identity matrices, and $\otimes$ denoting the Kronecker product you have met before.


In [ ]:
def kronecker_product(matrices):
    if len(matrices) == 1:
        return matrices[0]
    else:
        return np.kron(matrices[0], kronecker_product(matrices[1:]))

Id = np.matrix([[1.0, 0.0], [0.0, 1.0]])    # 2x2 Identity matrix
X = np.matrix([[0.0, 1.0], [1.0, 0.0]])     # Pauli X matrix
Y = np.matrix([[0.0, -1.0j], [1.0j, 0.0]])  # Pauli Y matrix
Z = np.matrix([[1.0, 0.0], [0.0, -1.0]])    # Pauli Z matrix

c_l = np.array([1.0, 0.2, 0.2]) # coefficients of the linear combination of A

A = c_l[0] * kronecker_product([Id, Id, Id]) + c_l[1] * kronecker_product([X, Z, Id]) + c_l[2] * kronecker_product([X, Id, Id])
print(A)

Also, we can map the vector $\bf b$ onto the quantum state $|b\rangle = H|0\rangle_{1} \otimes H|0\rangle_{2} \otimes H|0\rangle_{3}$.

In [ ]:
H = 1./np.sqrt(2)*np.matrix([[1.0, 1.0], [1.0, -1.0]]) # Hadamard matrix
ket_0 = np.array([1., 0.]) # |0>

b = kronecker_product([np.dot(H, ket_0), np.dot(H, ket_0), np.dot(H, ket_0)])
print(b)

The problem that we aim to solve on a quantum computer is that of preparing a quantum state $|x\rangle$, such that $A|x\rangle$ is proportional to $|b\rangle$.
 

### Solving linear systems with quantum computers

The possibility of solving linear systems on quantum computers has received considerable attention through the proposal of [Harrow-Hassidim-Lloyd](https://arxiv.org/abs/0811.3171) (HHL) originally in 2008, which is one of the main fundamental algorithms expected to provide an exponential speedup  over their classical counterparts (see https://en.wikipedia.org/wiki/HHL_algorithm). While the HLL quantum algorithm holds promise for the future, when large-scale fault-tolerant quantum computers exist, the timescale for such computers to become available remains an open question.

An interesting alternative strategy to solve systems of linear equations on currenlty available noisy itermediate-scale (NISQ) quantum computers is to employ a **variational** hybrid quantum-classical algorithm. These algorithms work by using the quantum computer for the part of the calculation that is hardest for the classical computer, while minimizing the total number of quantum operations as compared to doing everything on the quantum device.

### The Variational Quantum Linear Solver

The [Variational Quantum Linear Solver](https://arxiv.org/abs/1909.05820) (VQLS) takes two inputs

* a quantum gate sequence $U$ that prepares the quantum state $|b\rangle = U|0\rangle$ proportional to the vector $\bf b$
* a decomposition of the matrix $A$ into a linear combination $A = \sum_{l=1}^{L} c_l A_l$ of $L$ unitaries $A_l$ with complex coefficiencts $c_l$

to prepare a state $A|x\rangle \propto |b\rangle$. 

To solve the problem, VQLS employs an initial solution for the gate sequence $V({\bf \alpha})$ that prepares a potential solution $|x({\bf \alpha})\rangle = V({\bf \alpha})|0\rangle$. The parameters $\bf \alpha$ are classical inputs to a quantum computer which estimates a cost function $C({\bf \alpha})$ which quantifies the closeness of the quantum states $A|x(\alpha)\rangle$ and $|b\rangle$. The value of $C({\bf \alpha})$ is returned to a **classical** computer which iteratively adjusts $\bf \alpha$ in a classical optimization to minimize the cost. This continues until a termination condition is reached for $\alpha = \alpha_{\text{opt}}$. This interplay between classical and quantum operations is what makes the algorithm "hybrid". Finally, the output parameters $\alpha_{\text{opt}}$ can be used to prepare the solution state $|x(\alpha_{\text{opt}})\rangle = V(\alpha_{\text{opt}})|0\rangle$. 

<div align="center">
    <img src="images/vqls_flow.png" width="1000"/>
    <figcaption><a href="https://arxiv.org/abs/1909.05820">[arXiv:1909.05820]</a></figcaption>
</div>

#### Cost function

Several cost functions are discussed in the [original proposal](https://arxiv.org/abs/1909.05820) of VQLS. Here, we use a version favored by the authors for its advantages in trainability, that has been implemented in the [PennyLane VQLS tutorial](https://pennylane.ai/qml/demos/tutorial_vqls/#second-method):

$$
C_L = \frac{1}{2} - \frac{1}{2n} \frac{\sum_{j=0}^{n-1}\sum_{l,l^\prime}c_lc_{l^\prime}^\ast \mu_{l,l^\prime,j}}{\sum_{l,l^\prime}c_lc_{l^\prime}^\ast \mu_{l,l^\prime,-1}}
$$

with 

$$
\mu_{l,l^\prime,j} = \langle 0 | V^\dagger A_{l^\prime}^\dagger U Z_j U^\dagger A_l V | 0 \rangle
$$

and the convention $Z_{-1} = 1\!\!1$ if $j = -1$. These $\mu$ represent a **local** cost function on each qubit. The $\langle ... \rangle$ syntax means that we will compute an **inner product**, so that $\langle a | O | b \rangle = {\bf a}^\dagger \cdot O \cdot {\bf b} \in \mathbb{C}^0$.

The complex coefficients $\mu_{l,l^\prime,j}$ can be experimentally measured with a [Hadamard test](https://en.wikipedia.org/wiki/Hadamard_test). A Hadamard test is a method in quantum computation to obtain the real part $\operatorname{Re}(\langle\psi |O| \psi\rangle)$ or the imaginary part $\operatorname{Im}(\langle\psi |O| \psi\rangle)$ of an inner product  (where $|\psi\rangle$ is a multi-qubit quantum state and $O$ a unitary operation) by measuring an expectation value on only one additional so-called **ancilla** (helper) qubit. To do so, we just have to add two Hadarmard gates to the ancilla qubit and create a controlled version $\text{C}O$ from our unitary $O$ with the ancilla being the control qubit.

<div align="center">
    <img src="images/hadamard_test_re.png" width="350"/>
    <img src="images/hadamard_test_im.png" width="400"/>
    <figcaption>Hadamard test circuits to obtain $\operatorname{Re}(\langle\psi |O| \psi\rangle)$ (left) and $\operatorname{Im}(\langle\psi |O| \psi\rangle)$ (where $|\psi\rangle$ (right). </figcaption>
</div>
</br> 

Note that to measure the imaginary part, we have to rotate the ancilla qubit by $-\frac{\pi}{2}$ around the $z$-axis of the Bloch sphere which we can do with the phase shift operator $R_\phi(\phi)$ introduced in the previous section, which is for $\phi=-\frac{\pi}{2}$ equivalent to $S^\dagger = \begin{pmatrix}1 & 0 \\ 0 & -i\end{pmatrix}.$

Measuring the coefficients $\mu_{l,l^\prime,j}$ with Hadamard tests determines the actual circuits we need to implement next.

### Quantum circuit implementation

Our example problem introduced above is a linear system described by an $8 \times 8$ matrix and an $8$-element vector, which we can map onto a $3$-qubit quantum state. To measure the coefficients with a Hadarmard test, we need an additional ancilla qubit, such that we end up with a $4$-qubit quantum circuit implementation. We've seen above that we can decompose the matrix $A$ into $3$ unitaries $A_l$.

To evaluate the cost function $C_L$ *once*, we need to execute several instances of quantum circuits: For each of the $9$ pairs $l,l^\prime$ we need to perform the $8$ Hadamard tests shown below.

<div align="center">

<figure>
<img src="images/hadamard_test_j0_re.png" width="350"/>
<img src="images/hadamard_test_j0_im.png" width="350"/>
<figcaption>
Quantum circuits to measure the real (left) and imaginary (right) part of $\mu_{l,l^\prime,(j=0)}$  
</figcaption>
</figure>

<figure>
<img src="images/hadamard_test_j1_re.png" width="350"/>
<img src="images/hadamard_test_j1_im.png" width="350"/>
<figcaption>
Quantum circuits to measure the real (left) and imaginary (right) part of $\mu_{l,l^\prime,(j=1)}$  
</figcaption>
</figure>
<figure>
<img src="images/hadamard_test_j2_re.png" width="350"/>
<img src="images/hadamard_test_j2_im.png" width="350"/>
<figcaption>
Quantum circuits to measure the real (left) and imaginary (right) part of $\mu_{l,l^\prime,(j=2)}$  
</figcaption>
</figure>
<figure>
<img src="images/hadamard_test_j-1_re.png" width="300"/>
<img src="images/hadamard_test_j-1_im.png" width="300"/>
<figcaption>
Quantum circuits to measure the real (left) and imaginary (right) part of $\mu_{l,l^\prime,(j=-1)}$  
</figcaption>
</figure>
</div>


### Let's solve the example problem with PennyLane

<div class="alert alert-block alert-success"> 
    <b>Reminder:</b> <a href="https://pennylane.ai/">PennyLane</a> is an open-source software framework for variational quantum computing and quantum differentiable programming that allows you to build and ‘train’ quantum algorithms, like you would for a neural network. PennyLane also provides plenty of learning resources and demos like the <a href="https://pennylane.ai/qml/demos/tutorial_vqls">PennyLane VQLS tutorial</a> we follow in this workshop.
</div>



In [ ]:
# Pennylane
import pennylane as qml

# Plotting
import matplotlib.pyplot as plt

# Some variables specific to the linear system we want to solve
n_qubits = 3  # Number of system qubits
tot_qubits = n_qubits + 1  # Addition of an ancillary qubit
ancilla_idx = n_qubits  # Index of the ancillary qubit (last position)

# Hyperparameters for training of the variational circuit
steps = 30  # Number of optimization steps
step_size = 0.8  # Learning rate
q_delta = 0.001  # Initial spread of random quantum weights
rng_seed = 0  # Seed for random number generator

<div class="alert alert-block alert-info">
    <b>Your task:</b> Implement the subroutine for $U_b$
</div>

In [ ]:
def U_b():
    """Unitary matrix rotating the ground state to the problem vector |b> = U_b |0>."""
    for idx in range(n_qubits):
        qml.Hadamard(wires=idx)

<div class="alert alert-block alert-success"> 
    <b>Hint:</b> Check the PennyLane documentation of supported <a href="https://docs.pennylane.ai/en/stable/introduction/operations.html">Quantum Operators</a> for an overview of the available gates you may need to complete your tasks.
</div>

Let's recall for our example problem $$A = \sum\limits_{l = 1}^{L}c_l A_l = \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} + 0.2 X\otimes Z \otimes \mathbb{I} + 0.2 X\otimes \mathbb{I} \otimes \mathbb{I}$$ with $$c_l = [1.0, 0.2, 0.2]$$ and the unitaries 
$$
\begin{align}
A_0 &= \mathbb{I} \otimes \mathbb{I} \otimes \mathbb{I} \\ 
A_1 &= X\otimes Z \otimes \mathbb{I} \\ 
A_2 &= X\otimes \mathbb{I} \otimes \mathbb{I}
\end{align}
$$

for which we need to create [controlled versions](#Controlled-gates). Remember $\otimes$ denotes the Kronecker product, giving for example 

$$
X\otimes Z = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} \otimes \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix} = \begin{pmatrix} 0&0&1&0\\ 0&0&0&-1\\ 1&0&0&0\\ 0&-1&0&0 \end{pmatrix}.
$$

<div class="alert alert-block alert-info">
    <b>Your task:</b> Implement the subroutine for the <i>controlled</i> versions of the unitary components $A_l$ of $A$
</div>

In [ ]:
c_l = np.array([1.0, 0.2, 0.2])

def ctrl_A_l(idx):
    """Controlled versions of the unitary components A_l of the problem matrix A."""
    if idx == 0: # controlled-A_0
        qml.ctrl(qml.I, control=(ancilla_idx))(wires=0)
        qml.ctrl(qml.I, control=(ancilla_idx))(wires=1)
        qml.ctrl(qml.I, control=(ancilla_idx))(wires=2)

    elif idx == 1: # controlled-A_1
        qml.ctrl(qml.X, control=(ancilla_idx))(wires=0)
        qml.ctrl(qml.Z, control=(ancilla_idx))(wires=1)
        qml.ctrl(qml.I, control=(ancilla_idx))(wires=2)

    elif idx == 2: # controlled-A_2
        qml.ctrl(qml.X, control=(ancilla_idx))(wires=0)
        qml.ctrl(qml.I, control=(ancilla_idx))(wires=1)
        qml.ctrl(qml.I, control=(ancilla_idx))(wires=2)

<div class="alert alert-block alert-success"> 
    <b>Hint:</b> Remember you have seen how to build a controlled operation from a unitary in the <a href="#Controlled-gates">previous section</a>. To make our lives easier, PennyLane comes with a built-in <a href="https://docs.pennylane.ai/en/stable/introduction/operations.html#operator-functions">operator function</a> doing exactly that. 
</div>

<div class="alert alert-block alert-info">
    <b>Your task:</b> Implement the variational subroutine that should generate the candidate solution state $|x\rangle = V(\alpha)|0\rangle$.
</div>

The first layer of this variational subroutine is a product of Hadamard gates preparing a balanced superposition of all basis states. After that, we apply just a single layer of qubit rotations $R_y(\alpha_0) \otimes R_y(\alpha_1) \otimes R_y(\alpha_2)$. For solving more complex problems, we suggest using more expressive circuits as, e.g., the PennyLane [`StronglyEntanglingLayers()`](https://docs.pennylane.ai/en/stable/code/api/pennylane.StronglyEntanglingLayers.html) template. With *more expressive* we mean that the circuit can create more possible quantum states and such is capable of exploring a larger portion of
the entire state space.

In [ ]:
def V(alphas):
    """Variational circuit mapping the ground state |0> to the ansatz state |x(alphas)>."""
    # We first prepare an equal superposition of all the states of the computational basis.
    for idx in range(n_qubits):
        qml.Hadamard(wires=idx)

    # For a minimal variational circuit we just apply a layer of single-qubit Ry rotations.
    for idx, element in enumerate(alphas):
        qml.RY(element, wires=idx)

<div class="alert alert-block alert-info">
    <b>Review</b> how your subroutines are used in the Hadamard test circuit implementation and compare it with the circuit diagrams above.
</div>

In [ ]:
device = qml.device("lightning.qubit", wires=tot_qubits)


@qml.qnode(device, interface="autograd")
def local_hadamard_test(alphas, l=None, lprime=None, j=None, part=None):
    """Describe parameters here"""

    # First Hadamard gate applied to the ancillary qubit.
    qml.Hadamard(wires=ancilla_idx)

    # For estimating the imaginary part of the coefficient "mu", we must add a "-i"
    # phase gate.
    if part == "Im":
        qml.PhaseShift(-np.pi / 2, wires=ancilla_idx)
    qml.Barrier(wires=range(tot_qubits), only_visual=True)

    # Variational circuit generating a guess for the solution vector |x>
    V(alphas)
    qml.Barrier(wires=range(tot_qubits), only_visual=True)

    # Controlled application of the unitary component A_l of the problem matrix A.
    ctrl_A_l(l)
    qml.Barrier(wires=range(tot_qubits), only_visual=True)

    # Adjoint of the unitary U_b associated to the problem vector |b>.
    # In this specific example Adjoint(U_b) = U_b.
    U_b()
    qml.Barrier(wires=range(tot_qubits), only_visual=True)

    # Controlled Z operator at position j. If j = -1, apply the identity.
    if j != -1:
        qml.ctrl(qml.Z, control=(ancilla_idx))(wires=j)
        qml.Barrier(wires=range(tot_qubits), only_visual=True)

    # Unitary U_b associated to the problem vector |b>.
    U_b()
    qml.Barrier(wires=range(tot_qubits), only_visual=True)

    # Controlled application of Adjoint(A_lprime).
    # In this specific example Adjoint(A_lprime) = A_lprime.
    ctrl_A_l(lprime)
    qml.Barrier(wires=range(tot_qubits), only_visual=True)

    # Second Hadamard gate applied to the ancillary qubit.
    qml.Hadamard(wires=ancilla_idx)

    # Expectation value of Z for the ancillary qubit.
    return qml.expval(qml.PauliZ(wires=ancilla_idx))

<div class="alert alert-block alert-info">
    <b>Your task:</b> Draw the <code>local_hadamard_test</code> circuit for the different values of <code>l</code>, <code>l_prime</code>, <code>j</code>, and <code>part</code> to verify your circuits match the diagrams shown above.
</div>

In [ ]:
print(qml.draw_mpl(local_hadamard_test)(alphas=["alpha_0", "alpha_1", "alpha_2"], l=0, lprime=1, j=1, part="Im"))

As described above, we can measure either the real or imaginary part of $\mu_{l,l^\prime,j}$ with a Hadamard test. Because we are interested in both, we need to run each Hadamard test twice, done by the following function:

In [ ]:
def mu(alphas, l=None, lprime=None, j=None):
    """Generates the coefficients to compute the "local" cost function C_L."""

    mu_real = local_hadamard_test(alphas, l=l, lprime=lprime, j=j, part="Re")
    mu_imag = local_hadamard_test(alphas, l=l, lprime=lprime, j=j, part="Im")

    return mu_real + 1.0j * mu_imag

Here, we finally implement the full cost function of our minimzation problem. We use the expression of $C_L$ in terms of the coefficients $\mu_{l,l^\prime,j}$ given above.

In [ ]:
def cost(alphas):
    """Local version of the cost function. Tends to zero when A|x> is proportional to |b>."""

    # numerator
    mu_sum = 0.0

    for l in range(0, len(c_l)):
        for lprime in range(0, len(c_l)):
            for j in range(0, n_qubits):
                mu_sum = mu_sum + c_l[l] * np.conj(c_l[lprime]) * mu(alphas, l, lprime, j)

    mu_sum = abs(mu_sum)

    # denominator
    norm = 0.0

    for l in range(0, len(c_l)):
        for lprime in range(0, len(c_l)):
            norm = norm + c_l[l] * np.conj(c_l[lprime]) * mu(alphas, l, lprime, -1)

    norm = abs(norm)

    # Cost function C_L
    return 0.5 - 0.5 * mu_sum / (n_qubits * norm)

#### Variational optimization

To minimize the cost function we use a gradient-descent optimizer.

In [ ]:
opt = qml.GradientDescentOptimizer(stepsize=step_size)

Next, initialize the variational weights with random parameters.

In [ ]:
np.random.seed(rng_seed)
alphas = q_delta * np.random.randn(n_qubits, requires_grad=True)
print(f"Initial parameters: {alphas}")

<div class="alert alert-block alert-success">
    Now, we perform the optimization loop.<br>
    <b>Note</b> that the next cell will take about 1:30 minutes to complete.
</div>

In [ ]:
%%time 
cost_history = []
for it in range(steps):
    alphas, cost_before_step = opt.step_and_cost(cost, alphas)
    print("Step {:2d}/{:2d}:  Cost_L = {:9.7f}".format(it+1, steps, cost_before_step))
    cost_history.append(cost_before_step)

print(f"\nOptimal alphas: {alphas}")

We plot the cost function with respect to the optimization steps. We remark that this is not an abstract mathematical quantity since it also represents a bound for the error between the generated state and the exact solution of the problem.

In [ ]:
plt.plot(cost_history, "g")
plt.ylabel("Cost function")
plt.xlabel("Optimization steps")
plt.show()

<div class="alert alert-block alert-info">
    <b>Your task:</b> Repeat the training with different values of <code>steps</code> and <code>step_size</code>. How does the result change?
</div>

#### Prepare the quantum solution state

With the values of our variational parameters $\alpha$ that we have optimized above, we can generate the quantum state $|x\rangle$. As we mentioned previously, we can't read the amplitudes (remember you have read about amplitudes in the [previous section](#Qubits-and-qubit-states), already) of the state vector. Let's assume we are only interested in the probability $\left|\langle i | x \rangle \right|^2$ of each computational basis state $|i\rangle$ given the solution state $|x\rangle$.

<div class="alert alert-block alert-success"> 
    <b>Remember</b> how our solution vector ${\bf x}$ is encoded onto the quantum state vector $|x\rangle$ of our 3-qubit system:
    $$
    |x\rangle = x_0 |000\rangle + x_1 |001\rangle + x_2 |010\rangle + x_3 |011\rangle + x_4 |100\rangle + x_5 |101\rangle + x_6 |110\rangle + x_7 |111\rangle
    $$
</br>    
Hence, when we measure a bit-string probability of a particular basis state, for example $|000\rangle$, we get the absolute squared of the component $x_0$ of vector ${\bf x}$:
    $$
    \left|\langle 000 | x \rangle \right|^2 = \left|x_0\right|^2
    $$
</div>

To measure the bit-string probabilities of **each** basis state, we prepare the solution state $|x(\alpha_{\text{opt}})\rangle = V(\alpha_{\text{opt}})|0\rangle$ using only the variational subroutine `V` we have already implemented and sample the probabilities for the computational basis states using the optimal parameters from our training.

<div class="alert alert-block alert-info">
    <b>Your task:</b> Complete the implementation of the <code>prepare_and_sample</code> function.
</div>

In [ ]:
n_shots = 10 ** 6  # Number of quantum measurements
dev_x = qml.device("lightning.qubit", wires=n_qubits, shots=n_shots)

@qml.qnode(dev_x, interface="autograd")
def prepare_and_sample(alphas):

    # Variational circuit generating a guess for the solution vector |x>
    V(alphas)

    # Probability of each computational basis state.
    return qml.probs()


q_probs = prepare_and_sample(alphas)
print("|<i|x>|^2 =", q_probs)

#### Compare with classical results

Our example problem has a small size and we can solve it in a classically as shown before without problem. So let's do this once again via matrix inversion and compare the results with our quantum solution. In order to compare the result with the measurements we have performed on $|x\rangle$ above, we have to normalize and square the solution vector $\bf{x}$ obtained classically.

In [ ]:
A_inv = np.linalg.inv(A)
x = np.dot(A_inv, b.T)
c_probs = (x / np.linalg.norm(x)) ** 2
print("||x||^2 =", c_probs)

Let's visualize the outcome. As you can see, the results of both solutions match well.

In [ ]:
x = np.arange(2**n_qubits)
labels = [f"$|x_{i}|^2$" for i in x]
width = 0.25

fig, ax = plt.subplots(layout="constrained")
ax.bar(x, c_probs.flatten(), width, label="classical solution")
ax.bar(x + width, q_probs, width, label="quantum solution")
ax.set_xticks(x + width / 2., labels)
ax.set_ylim(top=0.2)
ax.legend(loc='upper left')
plt.show()

We see that there are many output bitstrings with reasonable likelihood of being measured. This is because the quantum computer, even in the ideal case, explores many (**not** necessarily all!) possible states as the circuit runs, and even with a fault-tolerant quantum computer there would be some likelihood of measuring the "wrong" or non-optimal solution. These algorithms are not guaranteed to find the global optimum if run once. 

<div class="alert alert-block alert-info">
    <b>Question:</b> How does the result change when we use a smaller number for <code>n_shots</code>?
</div>

<div class="alert alert-block alert-info">
    <b>Question:</b> What could be reasons for the differences we can observe?
</div>

## Running in the cloud on Amazon Braket Hybrid Jobs

We were able to solve the above example linear system with size $8\times8$ locally in our notebook without interacting with Amazon Braket. This would not be possible if we wouldn't use a local quantum circuit simulator like PennyLane's [Lightning Qubit](https://docs.pennylane.ai/projects/lightning/en/v0.25.0/devices.html) device but [real quantum hardware or a managed simulator on Braket](https://amazon-braket-pennylane-plugin-python.readthedocs.io/en/latest/devices/braket_remote.html). But what is with the classical part of our algorithm? Do we want to continue running it interactively in this notebook if we are offloading quantum tasks to the cloud? And what if we want to scale up and test our algorithm with a larger linear system of equations which requires more classical compute capacity and results in longer overall execution time, as well? Finally, what if we want to make sure we can track and durably store different versions of our algorithm code and hyperparameter configurations to ensure reproduceablility for the experiments we run while we develop and benchmark our algorithm?

Amazon Braket provides such functionality in the cloud with [Amazon Braket Hybrid Jobs](https://docs.aws.amazon.com/braket/latest/developerguide/braket-what-is-hybrid-job.html).

<div align="center">
    <figure>
        <img src="images/hybrid_jobs.png" width="1000"/>
    </figure>
</div>

Amazon Braket Hybrid Jobs enables you to run hybrid quantum-classical algorithms requiring both classical AWS resources and quantum processing units conveniently in a high-performance cloud environment providing the following main benefits.

**Performance:** Amazon Braket Hybrid Jobs provides better performance than running hybrid algorithms from your own environment. While your job is running, it has priority access to the selected target QPU. Tasks from your job run ahead of other tasks queued on the device. This results in shorter and more predictable runtimes for hybrid algorithms. Amazon Braket Hybrid Jobs also supports parametric compilation. You can submit a circuit using free parameters and Braket compiles the circuit once, without the need to recompile for subsequent parameter updates to the same circuit, resulting in even faster runtimes.

**Convenience:** Amazon Braket Hybrid Jobs simplifies setting up and managing your compute environment and keeping it running while your hybrid algorithm runs. You just provide your algorithm script and select a quantum device (either a quantum processing unit or a simulator) on which to run. Amazon Braket waits for the target device to become available, spins up the classical resources, runs the workload in pre-built or self-defined container environments, returns the results to Amazon Simple Storage Service (Amazon S3), and releases the compute resources.

**Metrics:** Amazon Braket Hybrid Jobs provides on-the-fly insights into running algorithms and delivers customizable algorithm metrics in near real-time to Amazon CloudWatch and the Amazon Braket console so you can track the progress of your algorithms.

### Solving a larger system of linear equations using Hybrid Jobs

Let's experience how this works in action. In the script [`vqls_pl.py`](vqls_pl.py) we have implemented VQLS for the $1024\times1024$ linear system

$$
\begin{align}
A &= 1.0 \mathbb{I}_{\otimes 10} + 0.25 Z_2 \otimes X_5 \otimes X_9 + 0.25 * X_0 \otimes Z_1 \otimes X_4 \otimes X_8 \\
|b\rangle &= H_0 \otimes H_1 \otimes H_2 \otimes H_3 \otimes H_6 \otimes H_7 |0\rangle
\end{align}
$$

Execute the cell below to create a hybrid job on Amazon Braket which executes the script with a certain hyperparameter configuation on an [ml.m5.large](https://aws.amazon.com/ec2/instance-types/m5/) instance running the classical subroutine as well as a PennyLane Lightning simulator [embedded](https://aws.amazon.com/blogs/quantum-computing/using-embedded-simulators-in-amazon-braket-hybrid-jobs/) in the job container to perform the circuit evaluations.

In [ ]:
from braket.aws import AwsQuantumJob
from braket.jobs.config import InstanceConfig

job = AwsQuantumJob.create(
    device="local:pennylane/lightning.qubit",
    source_module="vqls_pl.py",
    job_name="vqls-job",
    hyperparameters={
        'n_shots': 1000,  # number of quantum measurements per circuit invocation
        'n_steps': 100,   # number of interations for training
        'step_size': 0.8, # learning rate of gradient-descend optimizer
        'q_delta': 0.001, # spread of initial random value for variational parameters
    },
    instance_config=InstanceConfig(instanceType='ml.m5.large')
)
print(f"{job.arn} {job.state()}")

<div class="alert alert-block alert-info">
    The job will take a couple of minutes to complete. While it is running you may <b>review</b> the code and verify that we have made only minimal modifications in order to read the hyperparameters we pass during job creation and to store the job results. Also, go to the <a href="https://us-east-1.console.aws.amazon.com/braket/home?region=us-east-1#/jobs">Hybrid Jobs console</a> to <b>monitor your job</b>.
</div>

The job's state follows the lifecycle `CREATED`, `QUEUED`, `RUNNING`, `COMPLETED` and will take a few minutes to complete. You can query the job state to track its progress, programmatically.

In [ ]:
print(f"{job.arn} {job.state()}")

While the job is still running, we can already retrieve the metrics we define with the `log_metric` statements in the code. 

<div class="alert alert-block alert-success">
    <b>Note</b> that you may have to execute the following cell more than once to receive the latest metrics.
</div>

In [ ]:
from braket.jobs.metrics_data.definitions import MetricType

import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(job.metrics(metric_type=MetricType.ITERATION_NUMBER))
df.sort_values(by=["iteration_number"], inplace=True)

fig, ax1 = plt.subplots(1, 1)
fig.set_figwidth(10)
fig.suptitle("Job metrics", size=16, y=0.99)

ax1.axhline(y=0, color='black', linestyle='--')
ax1.plot("iteration_number", "cost_function", data=df)
ax1.set_xlabel('iteration', fontsize=18)
ax1.set_ylabel('cost', fontsize=18)
fig.show()

After the job completed, we can retrieve the results. Note that we only print the non-zero probabilities.

In [ ]:
job_result = job.result()
q_probs = np.array(job_result['q_probs'])

x = np.arange(len(q_probs))
labels = np.array([f"|x_{i}|^2" for i in x])
width = 0.25

labels = labels[np.nonzero(q_probs)]
values = q_probs[np.nonzero(q_probs)]
x = np.arange(len(values))

fig, ax = plt.subplots(layout="constrained")
fig.set_size_inches(30, 10)
ax.bar(x, values, width)
ax.set_xticks(x, labels, rotation=60, fontsize=15)
ax.set_ylim(top=0.03)
ax.set_xlim(left=-width, right=len(values))
ax.yaxis.set_tick_params(labelsize=20)
plt.show()

print(dict(map(lambda k,v : (k.item(),v.item()) ,labels, values)))

<div class="alert alert-block alert-info">
    <b>Task:</b> Run a couple of experiments with different hyperparameter configurations. How do the number of shots affect execution time and quality of the results? Try it out with <code>n_shots</code> equal to 100, 1000, and 10000.
</div>

<div class="alert alert-block alert-success">
    <b>Note</b> that you can run multiple jobs in parallel to speed up your experiments.
</div>

Let's now review some job metadata.

In [ ]:
print(f"Create: {job.metadata()['createdAt']}")
print(f"Started: {job.metadata()['startedAt']}")
print(f"Ended: {job.metadata()['endedAt']}")
print()
print(f"Braket tasks: {job.result()['braket_tasks']}")
print(f"Circuit invocations: {job.result()['circuit_invocations']}")
print(f"Runtime of the job instance: {job.metadata()['billableDuration']} milliseconds")

<div class="alert alert-block alert-info">
    <b>Questions:</b> 
    <ul>
        <li>How long did the job run?</li>
        <li>How many quantum circuits did it process during execution?</li>
        <li>Do you know why we don't see any <code>Braket tasks</code> associated to this job?</li>
        <li>Finally, we don't we run this on a real quantum computer?</li>
    </ul>
</div>

## Conclusion

Congratulations for completing the workshop **QTC301 | Exploring hybrid quantum-classical computation with Amazon Braket**. In this notebook, you have learned how to code up the Variational Quantum Linear Solver algorithm with PennyLane and used it to solve an $8\times8$ system of linear equations. Then, you took it to another level and learned how to run the entire hybrid quantum-classical algorithm in the cloud using Amazon Braket Hybrid Jobs to solve a $1024\times1024$ system of linear equations.

To continue learning about Amazon Braket we encourage you to check out the following resources:
* [Amazon Braket Examples](https://github.com/amazon-braket/amazon-braket-examples): Our primary repository for code examples on GitHub with Jupyter notebooks covering introductory level code snippets, advanced service features and reference implementations of canonical routines and example algorithms.
* [Amazon Braket Labs](https://github.com/amazon-braket/amazon-braket-labs): A GitHub repository highlighting community contributions and community-driven projects such as frameworks, examples, experimental components, and tools built on top of Braket.
* [Amazon Braket Digital Course](https://explore.skillbuilder.aws/learn/learning_plan/view/1990/amazon-braket-knowledge-badge-readiness-path-amazon): Our self-service learning path helping you build the knowledge and technical skills to use Amazon Braket, including domain-specific content, knowledge checks, hands-on labs and a knowledge badge assessment delivered on demand on AWS Skill Builder.
* [AWS Quantum Technologies Blog Channel](https://aws.amazon.com/blogs/quantum-computing/): Stay in the know with new features, service announcements and news from the community and our customers and partners.
* [AWS Cloud Credit for Research](https://aws.amazon.com/government-education/research-and-technical-computing/cloud-credit-for-research): If you are a researcher interested to use the cloud for research workloads, apply here for credits to access technology that accelerates innovation.
* [Quantum Embark](https://aws.amazon.com/blogs/quantum-computing/aws-announces-the-quantum-embark-program-to-help-customers-get-ready-for-quantum-computing/): If you are a customer seeking trustworthy information about the relevance of quantum technology to your most important use cases, to understand the current state-of-play, and make informed decisions about when is the right time to engage, check out our Quantum Embark Program.

<div class="alert alert-block alert-success">
    <b>Remember</b> that we will have to terminate the workshop environments after this session. If you want to keep this notebook for your own reference, we recommend you create a zip archive to the QTC301 directory and download it on your laptop.
</div>

<div align="center">
<img src="images/thank_you.jpeg" width="600"/>
</div>